# 1. Activate GPU and Install Dependencies

In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub evaluate wandb -q
#!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import login
from google.colab import userdata


login(token = userdata.get('HUGGINGFACE_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#1. Load dataset

In [ ]:
#splits = {'train': 'data/train-00000-of-00001-7b34565378f02992.parquet', 'val': 'data/val-00000-of-00001-d7338c59b5e5031f.parquet', 'test': 'data/test-00000-of-00001-c830a979da438bff.parquet'}
#df_train = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["train"])

In [ ]:
ds = load_dataset("PrevenIA/spanish-suicide-intent")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

(…)-00000-of-00001-7b34565378f02992.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

(…)-00000-of-00001-d7338c59b5e5031f.parquet:   0%|          | 0.00/2.03M [00:00<?, ?B/s]

(…)-00000-of-00001-c830a979da438bff.parquet:   0%|          | 0.00/4.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/136136 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/15131 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/37820 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset

# Filtrar los ejemplos con label 0 y 1
label_0_train = ds["train"].filter(lambda x: x['Label'] == 0)
label_1_train = ds["train"].filter(lambda x: x['Label'] == 1)

label_0_test = ds["test"].filter(lambda x: x['Label'] == 0)
label_1_test = ds["test"].filter(lambda x: x['Label'] == 1)

# Definir el tamaño del conjunto balanceado
train_size = 50000
test_size = 5000

# Seleccionar aleatoriamente ejemplos de cada etiqueta
selected_train_0 = label_0_train.shuffle(seed=42).select([i for i in range(train_size)])
selected_train_1 = label_1_train.shuffle(seed=42).select([i for i in range(train_size)])

selected_test_0 = label_0_test.shuffle(seed=42).select([i for i in range(test_size)])
selected_test_1 = label_1_test.shuffle(seed=42).select([i for i in range(test_size)])

# Combinar los ejemplos de label 0 y label 1
combined_train = Dataset.from_dict({
    key: selected_train_0[key] + selected_train_1[key]
    for key in selected_train_0.column_names
})

combined_test = Dataset.from_dict({
    key: selected_test_0[key] + selected_test_1[key]
    for key in selected_test_0.column_names
})

# Barajar nuevamente para asegurar la aleatoriedad
small_train_dataset = combined_train.shuffle(seed=42)
small_test_dataset = combined_test.shuffle(seed=42)

# Verifica los primeros elementos
print(small_train_dataset[0])
print(small_test_dataset[0])

{'Text': 'Tú me pides un deseo, y yo trato de encontrar una manera de concedértelo técnicamente, pero mske lo más mierda posible, Fullllllllllllllllllllllllllllllllllllllllllllllllll', 'Label': 0, 'dataset': 'SuicideWatch', '__index_level_0__': 73114}
{'Text': 'Acabo de ver uno de esos camiones peapod y me reí a carcajadas porque decía "naranja, te alegra que te lo entreguemos".\r\n\r\nQué está mal conmigo', 'Label': 1, 'dataset': 'TwitterSuicidalAnalysis', '__index_level_0__': 2771}


In [ ]:
small_train_dataset = small_train_dataset.rename_column("Label", "labels")
small_test_dataset = small_test_dataset.rename_column("Label", "labels")

# Preprocess

In [ ]:
# Set DistilBERT tokenizer

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["Text"], truncation=True)

tokenized_train = tokenized_imdb = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluate

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
#clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
#clf_metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# 3. Training the model

In [ ]:
#id2label = {0: "normal", 1: "intencion"}
#label2id = {"normal": 0, "intencion": 1}

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, #id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-bert-text-classification"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    #remove_unused_columns=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
from datetime import datetime
import wandb
today = datetime.today()
run_date = today.strftime("%d-%m-%Y_%H_%M")

wb_token = userdata.get('wandb')

wandb_config = {
    "output_dir": repo_name,
    "learning_rate": training_args.learning_rate,
    "per_device_train_batch_size": training_args.per_device_train_batch_size,
    "per_device_eval_batch_size": training_args.per_device_eval_batch_size,
    "num_train_epochs": training_args.num_train_epochs,
    "weight_decay": training_args.weight_decay,
    "eval_strategy": training_args.eval_strategy,
    "save_strategy": training_args.save_strategy,
    "load_best_model_at_end": training_args.load_best_model_at_end,
    "push_to_hub": training_args.push_to_hub
}

wandb.login(key=wb_token)
run = wandb.init(
    project='finetuning-bert-text-classification on suicide intent-spanish',
    job_type="training",
    name=f"experiment_finetuning_bert_{run_date}",
      # Track hyperparameters and run metadata
    config=wandb_config,
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.261200,0.239404,0.907300
2,0.196600,0.248769,0.918400
3,0.173700,0.275917,0.919200
4,0.141500,0.332236,0.916500
5,0.085700,0.383476,0.919900


TrainOutput(global_step=31250, training_loss=0.18319126763916016, metrics={'train_runtime': 2617.7393, 'train_samples_per_second': 191.005, 'train_steps_per_second': 11.938, 'total_flos': 2.7233711694495936e+16, 'train_loss': 0.18319126763916016, 'epoch': 5.0})

In [ ]:
# Compute the evaluation metrics
metrics = trainer.evaluate()

In [ ]:
metrics

{'eval_loss': 0.23940439522266388,
 'eval_accuracy': 0.9073,
 'eval_runtime': 17.3129,
 'eval_samples_per_second': 577.605,
 'eval_steps_per_second': 36.1,
 'epoch': 5.0}

In [ ]:
wandb.log({"Accuracy": metrics['eval_accuracy']})
wandb.log({"metrics": metrics})

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()

events.out.tfevents.1727392408.1dce0b1b21b8.1044.5:   0%|          | 0.00/417 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1727389752.1dce0b1b21b8.1044.4:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AndresR2909/finetuning-bert-text-classification/commit/2c5352111ad5940a85b5b75eb3ac9ae77e5439bb', commit_message='End of training', commit_description='', oid='2c5352111ad5940a85b5b75eb3ac9ae77e5439bb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
wandb.finish()
model.config.use_cache = True

Accuracy,▁
eval/accuracy,▁▇█▆█▁
eval/loss,▁▁▃▆█▁
eval/runtime,██▇▅▃▁
eval/samples_per_second,▁▁▂▄▆█
eval/steps_per_second,▁▁▂▄▆█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████
train/grad_norm,▂▃▂▃▁▄▃▁▄▃▃▃▃▄▂▃▃▁▂▅▄▁▂▃▁▅▃▂▅▁▄█▁▁▅▁▃▁▄▁
train/learning_rate,███▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
train/loss,█▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▂▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁


# 4. Analyzing new data with the model

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="AndresR2909/finetuning-bert-text-classification",  truncation=True,)

sentiment_model(["I love this move", "This movie sucks!"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.6995328664779663},
 {'label': 'LABEL_0', 'score': 0.8990325331687927}]

In [ ]:
import pandas as pd
splits = {'train': 'data/train-00000-of-00001-7b34565378f02992.parquet', 'val': 'data/val-00000-of-00001-d7338c59b5e5031f.parquet', 'test': 'data/test-00000-of-00001-c830a979da438bff.parquet'}
df_train = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["train"])

In [ ]:
# Filtra los datos con Label igual a 1
df_label_1 = df_train[df_train['Label'] == 1]

# Filtra los datos con Label igual a 0
df_label_0 = df_train[df_train['Label'] == 0]

# Toma 10,000 muestras aleatorias de cada grupo
sampled_label_1 = df_label_1.sample(n=10000, random_state=42)
sampled_label_0 = df_label_0.sample(n=10000, random_state=42)

# Combina ambos DataFrames
sampled_df = pd.concat([sampled_label_1, sampled_label_0])

# Reordena las filas aleatoriamente

df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
df.Label.value_counts()

,count
Label,
0,10000
1,10000


In [ ]:
# Split the DataFrame
train_size = 0.8
eval_size = 0.1

# Calculate sizes
train_end = int(train_size * len(df))
eval_end = train_end + int(eval_size * len(df))

# Split the data
X_train = df[:train_end]
X_eval = df[train_end:eval_end]
X_test = df[eval_end:]

In [ ]:
X_test

,Text,Label,dataset,__index_level_0__
18000,Estoy listo para morir.2 años de tiempo presta...,1,SuicideWatch,36253
18001,Creé un avatar reddit e hice que se pareciera ...,0,SuicideWatch,55987
18002,estos son los niños que luego en la vida están...,1,TwitterSuicidalAnalysis,10161
18003,"Estoy trabajando ahora, el primer día de mi nu...",1,SuicideWatch,93875
18004,Pensamientos matutinos Hubo un tiempo en el qu...,1,Ji et al. 2018,1764
...,...,...,...,...
19995,Mi escuela publicó esto a los estudiantes idk ...,0,SuicideWatch,44457
19996,Esto es un poco grande para mí porque he estad...,0,SuicideWatch,2270
19997,¿Estarías dispuesto a sacarme de la cornisa? M...,1,SuicideWatch,108633
19998,Me estoy ahogando lentamente por los números,1,TwitterSuicidalAnalysis,25724


In [ ]:
y  = sentiment_model(X_test['Text'].to_list(),  truncation=True)

In [ ]:
y_pred = [int(pred['label'][-1]) for pred in y]
len(y_pred)

2000

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = X_test['Label'].tolist()

print("Classification Report:")
print(classification_report(y_true, y_pred))

# Paso 5: Generar la matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred)

# Convertir la matriz de confusión en un DataFrame para facilitar la visualización
conf_matrix_df = pd.DataFrame(conf_matrix,
                              index=['True 0', 'True 1'],
                              columns=['Pred 0', 'Pred 1'])

print("Confusion Matrix:")
print(conf_matrix_df)

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       994
           1       0.91      0.93      0.92      1006

    accuracy                           0.92      2000
   macro avg       0.92      0.92      0.92      2000
weighted avg       0.92      0.92      0.92      2000

Confusion Matrix:
        Pred 0  Pred 1
True 0     901      93
True 1      66     940
